In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import cartopy.crs as ccrs
import gpflow
import tensorflow as tf
import glob
from utils_GP import *

will turn into GP2: March 2024, adding geodesic distance metric following geokernels (https://github.com/sigmaterra/geokernels) which is an implimentation for sklearn kernels.
